In [471]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os. walk('/kaggle/input'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

In [472]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [473]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
gender_df = pd.read_csv("gender_submission.csv")

test_PassengerId = test_df.PassengerId,train_df.columns
combine = [train_df,test_df]

In [474]:
gender_df.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [475]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [476]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [477]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [478]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [479]:
pd.crosstab(train_df['Pclass'], train_df['Survived'], margins=True)

Survived,0,1,All
Pclass,,,
1,80,136,216
2,97,87,184
3,372,119,491
All,549,342,891


In [480]:
pd.crosstab(train_df['Age'], train_df['Survived'], margins=True)

Survived,0,1,All
Age,,,
0.42,0,1,1
0.67,0,1,1
0.75,0,2,2
0.83,0,2,2
0.92,0,1,1
...,...,...,...
70.5,1,0,1
71.0,2,0,2
74.0,1,0,1


In [481]:
pd.crosstab(train_df['Fare'], train_df['Survived'], margins=True)

Survived,0,1,All
Fare,,,
0.0,14,1,15
4.0125,1,0,1
5.0,1,0,1
6.2375,1,0,1
6.4375,1,0,1
...,...,...,...
247.5208,1,1,2
262.375,0,2,2
263.0,2,2,4


In [482]:
pd.crosstab(train_df['Sex'], train_df['Survived'], margins=True)

Survived,0,1,All
Sex,,,
female,81,233,314
male,468,109,577
All,549,342,891


In [483]:
pd.crosstab(train_df['Embarked'], train_df['Survived'], margins=True)

Survived,0,1,All
Embarked,,,
C,75,93,168
Q,47,30,77
S,427,217,644
All,549,340,889


In [484]:
pd.crosstab(train_df['Fare'], train_df['Pclass'], margins=True)

Pclass,1,2,3,All
Fare,,,,
0.0,5,6,4,15
4.0125,0,0,1,1
5.0,1,0,0,1
6.2375,0,0,1,1
6.4375,0,0,1,1
...,...,...,...,...
247.5208,2,0,0,2
262.375,2,0,0,2
263.0,4,0,0,4


In [485]:
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1 
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

In [486]:
pd.crosstab(train_df['FamilySize'], train_df['Survived'], margins=True)

Survived,0,1,All
FamilySize,,,
1,374,163,537
2,72,89,161
3,43,59,102
4,8,21,29
5,12,3,15
6,19,3,22
7,8,4,12
8,6,0,6
11,7,0,7


In [487]:
train_df['Nuclear']=(2<=train_df['FamilySize']) & (train_df['FamilySize']<=4)

test_df['Nuclear']=(2<=test_df['FamilySize']) & (test_df['FamilySize']<=4)

In [488]:
train_df['Embarked'].fillna('S', inplace=True)

In [489]:
train_df.drop(['SibSp', 'Parch', 'Ticket','Cabin'], axis=1, inplace=True)
test_df.drop(['SibSp', 'Parch', 'Ticket','Cabin'], axis=1, inplace=True)

In [490]:
train_df['Sex'].replace(['male','female'],
                        ['0','1'],inplace=True)
test_df['Sex'].replace(['male','female'],
                        ['0','1'],inplace=True)

In [491]:
train_df['Fare26-']=(train_df['Fare'] > 52.5)
test_df['Fare26-']=(test_df['Fare'] > 52.5)

train_df=train_df.drop(columns='Fare')
test_df=test_df.drop(columns='Fare')

In [492]:
train_df['Name']=train_df['Name'].str.split(', ').str[1].str.split('. ').str[0]
test_df['Name']=test_df['Name'].str.split(', ').str[1].str.split('. ').str[0]

train_df['Name'].head()

0      Mr
1     Mrs
2    Miss
3     Mrs
4      Mr
Name: Name, dtype: object

In [493]:
train_df['Age0-15'] = (train_df['Age'] < 16) & (train_df['Age'] != 2)
test_df['Age0-15'] = (test_df['Age']  < 16) & (test_df['Age'] != 2)

In [494]:
train_df['Master']=(train_df['Name']=='Master')
test_df['Master']=(test_df['Name']=='Master')
train_df['Mrs']=(train_df['Name']=='Mrs')
test_df['Mrs']=(test_df['Name']=='Mrs')
train_df['Miss']=(train_df['Name']=='Miss')
test_df['Miss']=(test_df['Name']=='Miss')

train_df.drop(['Name'], axis=1, inplace=True)
test_df.drop(['Name'], axis=1, inplace=True)

In [495]:
train_df['Pclass_3']=(train_df['Pclass']==3)
train_df['Pclass_1']=(train_df['Pclass']==1)

test_df['Pclass_3']=(test_df['Pclass']==3)
test_df['Pclass_1']=(test_df['Pclass']==1)

train_df=train_df.drop(columns='Pclass')
test_df=test_df.drop(columns='Pclass')

In [496]:
train_df['EmbarkedQ']=train_df['Embarked']=='Q'
train_df['EmbarkedS']=train_df['Embarked']=='S'

test_df['EmbarkedS']=test_df['Embarked']=='S'
test_df['EmbarkedQ']=test_df['Embarked']=='Q'

train_df=train_df.drop(columns='Embarked')
test_df=test_df.drop(columns='Embarked')

In [497]:
train_df.drop(['PassengerId'], axis=1, inplace=True)

In [498]:
train_df.drop(['FamilySize','Age'], axis=1, inplace=True)
test_df.drop(['FamilySize','Age'], axis=1, inplace=True)

In [499]:
train_df.isnull().sum()

Survived     0
Sex          0
Nuclear      0
Fare26-      0
Age0-15      0
Master       0
Mrs          0
Miss         0
Pclass_3     0
Pclass_1     0
EmbarkedQ    0
EmbarkedS    0
dtype: int64

In [500]:
test_df.isnull().sum()

PassengerId    0
Sex            0
Nuclear        0
Fare26-        0
Age0-15        0
Master         0
Mrs            0
Miss           0
Pclass_3       0
Pclass_1       0
EmbarkedS      0
EmbarkedQ      0
dtype: int64

In [501]:
X_train = train_df.drop("Survived", axis=1)

Y_train = train_df["Survived"]

X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 11), (891,), (418, 11))

In [502]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred0 = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

80.81

In [503]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
0,Sex,2.191674
4,Master,1.802524
8,Pclass_1,0.693194
5,Mrs,0.636307
3,Age0-15,0.474961
6,Miss,0.377990
2,Fare26-,0.350404
1,Nuclear,0.261702
9,EmbarkedQ,0.062358
10,EmbarkedS,-0.497169


In [504]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred1 = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

84.29

In [505]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred2 = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

82.72

In [506]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred3 = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

77.44

In [507]:
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred4 = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

77.44

In [508]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred5 = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
print(acc_linear_svc)

80.81


In [509]:
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred6 = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

80.7

In [510]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred7 = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

84.4

In [511]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

84.4

In [512]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,84.40
8,Decision Tree,84.40
0,Support Vector Machines,84.29
1,KNN,82.72
2,Logistic Regression,80.81
7,Linear SVC,80.81
6,Stochastic Gradient Decent,80.70
4,Naive Bayes,77.44
5,Perceptron,77.44


In [513]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred1
    })

In [514]:
submission.to_csv('submission.csv',index = False)

In [515]:
k = pd.read_csv('submission.csv')

In [516]:
k.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
